<font size = 8 color ='27A3D7'>Dashboard del Chatbot</font>

---

Este dashboard debe ser desarrollado usando streamlit, y debe funcionar por filtros dependiento la campaña que se ocupe, el usuario debe tener la posibilidad de modificar los filtros de acuerdo a la necesidad.

#### Reporte: https://docs.google.com/document/d/1n3Kw6Y5ToY3OntV1OtoagU3tRSi7315E/edit

### Paleta de Colores

---

<span style="color:#06417C;">Texto en #06417C</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:#145CB3;">Texto en #145CB3</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:#27A3D7;">Texto en #27A3D7</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:#EDF3F9;">Texto en #EDF3F9</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:#F7A94B;">Texto en #F7A94B</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:#56070C;">Texto en #56070C</span>


<span style="color:rgb(6, 65, 124);">Texto en RGB(6, 65, 124)</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:rgb(20, 92, 179);">Texto en RGB(20, 92, 179)</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:rgb(39, 163, 215);">Texto en RGB(39, 163, 215)</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:rgb(237, 243, 249);">Texto en RGB(237, 243, 249)</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:rgb(247, 169, 75);">Texto en RGB(247, 169, 75)</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:rgb(86, 7, 12);">Texto en RGB(86, 7, 12)</span>


## Autenticación

---

- Ponerles login: Inicio de Streamlit, el repo publico pero la app no. 

---

## Documentación



| **Attribute Name**   | **Attribute Type** | **Description**                                                                 |
|----------------------|--------------------|---------------------------------------------------------------------------------|
| `product`            | string             | Nombre del producto asociado al contacto, como una tarjeta de crédito.          |
| `created_at`         | string             | Fecha en la que se creó este registro.                                          |
| `contact_phone`      | string             | Número de teléfono del contacto.                                                |
| `bot_id`             | string             | Identificador en RDS del bot al que pertenece el contacto.                      |
| `status`             | string             | Estado del contacto en relación a las contactaciones: `fc` (primer contacto), `ws` (sin estado), `bs` (antes del estado), `rc` (reintento de contacto). |
| `enterprise_id`      | string             | ID en RDS de la empresa a la que pertenece el contacto: 3. Monte 2. BanCoppel   |
| `last_message_at`    | string             | Fecha y hora del último mensaje recibido en la conversación.                    |
| `unseen_messages`    | boolean            | Indicador de si hay mensajes pendientes no vistos del contacto.                 |
| `discount`           | string             | Porcentaje de descuento autorizado al contacto para liquidar.                   |
| `last_template_at`   | string             | Fecha y hora en que se envió el último mensaje de plantilla.                    |
| `total_balance`      | float              | Monto de dinero adeudado por el contacto.                                       |
| `credit`             | string             | Si aplica, número de crédito del contacto.                                      |
| `user_id`            | string             | ID de la cuenta del agente que está chateando con este contacto (obtenido de AWS Cognito). |
| `template_id`        | string             | Plantilla seleccionada para la contactación cuando se subió el contacto.        |
| `last_name`          | string             | Apellido del contacto.                                                          |
| `messages`           | map                | Objeto JSON en forma de cadena que contiene el historial de mensajes entre el usuario y el bot/agente. |
| `first_name`         | string             | Nombre del contacto.                                                            |
| `retry_count`        | integer            | Contador para registrar los intentos de contacto.                               |
| `session_id`         | string             | Identificador de la API Meta Cloud del número de teléfono.                      |
| `bot_enabled`        | boolean            | Indicador que determina si el bot responderá el chat si se recibe una respuesta. |
| `tag`                | string             | Etiqueta para definir el estado de la conversación (usado para clasificación).   |
| `error`              | string             | Etiqueta para...                                                                 |





Link de documentación: https://www.notion.so/pernexium/Data-Bases-Documentation-766c6f74424e45bcb10c5d290161a9f1

## Mensajes en Meta

---


Status de los mensajes: https://developers.facebook.com/docs/whatsapp/on-premises/webhooks/outbound/?locale=es_LA

| Nombre   | Descripción                                                                                                                                                                                      |
|----------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| deleted  | El usuario eliminó un mensaje que envió. Cuando recibas esta notificación, deberás eliminar el mensaje del sistema si se descargó previamente desde el servidor.                                   |
| delivered| El dispositivo del usuario recibió el mensaje enviado por tu empresa.                                                                                                                              |
| failed   | No se pudo enviar el mensaje de tu empresa. Se incluirá el motivo del error en la devolución de llamada. Consulta la documentación sobre mensajes de error para obtener ayuda con la depuración:   |
|          | - Errores de la API de instalaciones locales                                                                                                                                                       |
|          | - Códigos de error y Solución de problemas de la API de la nube                                                                                                                                    |
| read     | El usuario leyó un mensaje que envió tu empresa. Las notificaciones read solo están disponibles para los usuarios que tengan activadas las confirmaciones de lectura. Los usuarios que no las tengan activadas solo recibirán la notificación delivered. |
| sent     | Un mensaje enviado por tu empresa se encuentra en tránsito dentro de nuestros sistemas.                                                                                                             |
|          | Usuarios de la API de instalaciones locales: para recibir notificaciones de mensajes sent, es necesario definir sent_status como true en la configuración de la app. La notificación de estado sent está desactivada de manera predeterminada.          |
| warning  | Un mensaje enviado por tu empresa contiene un artículo de un catálogo que no está disponible o no existe.                                                                                           |


### Importamos librerías

---

In [562]:
import json
import boto3
import random
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from io import BytesIO
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

## Importamos la base de BanCoppel desde S3

---

In [563]:



def create_aws_session():
    session = boto3.Session(
        aws_access_key_id = AWS_ACCESS_KEY_ID,
        aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
        region_name = AWS_REGION
    )
    return session

In [564]:
session = create_aws_session()
s3_client = session.client('s3')

current_date = datetime.now().strftime('%Y_%m')
bucket_name = 's3-pernexium-report'
prefix = f'raw/bancoppel/replica_dynamo_chatbot/{current_date}/'
response = s3_client.list_objects_v2(Bucket = bucket_name, Prefix = prefix)
files = response.get('Contents', [])
files_sorted = sorted(files, key = lambda x: x['LastModified'], reverse = True)
latest_file = files_sorted[0]['Key']
response = s3_client.get_object(Bucket = bucket_name, Key = latest_file)
content = response['Body'].read()
df_bancoppel = pd.read_json(BytesIO(content), lines = True)

In [565]:
json_data = json.loads(content)
if isinstance(json_data, list):
    df_bancoppel = pd.DataFrame(json_data)
else:
    df_bancoppel = pd.json_normalize(json_data)

In [566]:
print(df_bancoppel.shape)

(27494, 22)


In [567]:
df_bancoppel.head(3)

,product,created_at,contact_phone,bot_id,status,enterprise_id,last_message_at,unseen_messages,discount,last_template_at,...,user_id,template_id,last_name,messages,first_name,retry_count,session_id,bot_enabled,tag,error
0,TDC VISA,2024-07-26T18:41:58.692Z,5212201140267,13c046be-c642-490a-bf8b-92206ca5a54d,rc,2,1970-01-01T00:00:00.000Z,False,0,2024-08-22T15:58:31.325Z,...,015bf5b0-0021-70bd-02d0-8a3d232785c2,banco_nos_interesa_acuerdo,JOSE,"[{'createdAt': '2024-07-26T18:44:10.501Z', 'role': 'assistant', 'content': 'Template ""banco_liquidacion_3_1"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-07-26T18:44:10.501Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjIwOTYxNjM5RjVGMDNFNDMxNwA=', 'content': 'Aprovecha esta oportunidad que BanCoppel tiene para ti, para que liquides tu cuenta, tenemos opciones para ti, estamos para apoyarte.', 'direction': 'outgoing', 'status': 'failed'}, {'createdAt': '2024-08-22T15:58:31.325Z', 'role': 'assistant', 'content': 'Template ""banco_nos_interesa_acuerdo"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-08-22T15:58:31.325Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjI5QkZDQjQ2Q0VFRUE0NkFDNgA=', 'content': 'Nos interesa llegar a un acuerdo para tu credito *BanCoppel*, estamos para ayudarte, ¿qué opinas?', 'direction': 'outgoing', 'status': 'failed'}]",MARIA,1,353257377876857,True,NaN,NaN
1,TDC VISA,2024-08-01T00:02:52.340Z,5212211180442,13c046be-c642-490a-bf8b-92206ca5a54d,rc,2,1970-01-01T00:00:00.000Z,False,0,2024-08-10T14:48:24.586Z,...,015bf5b0-0021-70bd-02d0-8a3d232785c2,banco_tenemos_alternativas,FABIOLA,"[{'createdAt': '2024-08-01T00:07:50.281Z', 'role': 'assistant', 'content': 'Template ""banco_como_te_ayudamos"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-08-01T00:07:50.281Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIxMTE4MDQ0MhUCABEYEjY4MDQ5NEQxRUIyOUI1M0RDMwA=', 'content': 'Estimado cliente de *BanCoppel* queremos ayudarte para que liquides tu credito, ¿cómo te podemos apoyar?', 'direction': 'outgoing', 'status': 'failed'}, {'createdAt': '2024-08-10T14:48:24.586Z', 'role': 'assistant', 'content': 'Template ""banco_tenemos_alternativas"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-08-10T14:48:24.586Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIxMTE4MDQ0MhUCABEYEkI3NkUzNzM3QUU3QzFDMkJCNQA=', 'content': '¡Excelente día! tenemos muy buenas alternativas para que liquides tu cuenta con *BanCoppel*, esperamos tu respuesta.', 'direction': 'outgoing', 'status': 'failed'}]",GLENDA,1,353257377876857,True,NaN,NaN
2,TDC VISA,2024-08-22T15:52:01.079Z,5212211256467,13c046be-c642-490a-bf8b-92206ca5a54d,fc,2,1970-01-01T00:00:00.000Z,False,0,2024-08-22T15:55:08.891Z,...,015bf5b0-0021-70bd-02d0-8a3d232785c2,banco_nos_interesa_acuerdo,TLAHUEL,"[{'createdAt': '2024-08-22T15:55:08.891Z', 'role': 'assistant', 'content': 'Template ""banco_nos_interesa_acuerdo"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-08-22T15:55:08.891Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIxMTI1NjQ2NxUCABEYEkMyNUFCNkYzMDUxQzNERDczQQA=', 'content': 'Nos interesa llegar a un acuerdo para tu credito *BanCoppel*, estamos para ayudarte, ¿qué opinas?', 'direction': 'outgoing', 'status': 'read'}]",ENRIQUE,0,353257377876857,True,NaN,NaN


{**'createdAt'**: '2024-08-07T16:36:10.054Z', **'role'**: 'assistant', **'id'**: 'wamid.HBgNNTIxMjIxMTM4NTI0NxUCABEYEkYzOUVGMEY1NzEzNzA4N0JDNgA=', **'content'**: 'Bancoppel te agradece la liquidación de tu saldo vencido, o bien pregunta por las alternativas que tenemos para ti, esperamos tu pronta respuesta', **'direction':** 'outgoing', **'status':** 'failed'}]
Name: messages, dtype: object

In [568]:
df_bancoppel.columns

Index(['product', 'created_at', 'contact_phone', 'bot_id', 'status',
       'enterprise_id', 'last_message_at', 'unseen_messages', 'discount',
       'last_template_at', 'total_balance', 'credit', 'user_id', 'template_id',
       'last_name', 'messages', 'first_name', 'retry_count', 'session_id',
       'bot_enabled', 'tag', 'error'],
      dtype='object')

## Análisis de Datos: Respondiento preguntas para BanCoppel

---

In [569]:
df_bancoppel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27494 entries, 0 to 27493
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product           27493 non-null  object
 1   created_at        27494 non-null  object
 2   contact_phone     27494 non-null  object
 3   bot_id            27494 non-null  object
 4   status            27494 non-null  object
 5   enterprise_id     27494 non-null  object
 6   last_message_at   27494 non-null  object
 7   unseen_messages   27494 non-null  bool  
 8   discount          27494 non-null  object
 9   last_template_at  27494 non-null  object
 10  total_balance     27494 non-null  object
 11  credit            27494 non-null  object
 12  user_id           27494 non-null  object
 13  template_id       27494 non-null  object
 14  last_name         27494 non-null  object
 15  messages          27494 non-null  object
 16  first_name        27494 non-null  object
 17  retry_count 

In [570]:
df_bancoppel.describe()

,product,created_at,contact_phone,bot_id,status,enterprise_id,last_message_at,unseen_messages,discount,last_template_at,...,user_id,template_id,last_name,messages,first_name,retry_count,session_id,bot_enabled,tag,error
count,27493,27494,27494,27494,27494,27494,27494,27494,27494,27494,...,27494,27494,27494,27494,27494,27494,27494,27494,1758,2
unique,5,26888,25766,2,3,1,3407,2,2,27494,...,8,14,3678,27494,2889,12,1,2,9,1
top,TDC VISA,2024-08-07T15:20:19.591Z,5216648057935,13c046be-c642-490a-bf8b-92206ca5a54d,rc,2,1970-01-01T00:00:00.000Z,False,0,2024-08-22T15:58:31.325Z,...,311b05e0-60a1-70e9-63dc-771c224b6735,banco_como_te_ayudamos,GUADALUPE,"[{'createdAt': '2024-07-26T18:44:10.501Z', 'role': 'assistant', 'content': 'Template ""banco_liquidacion_3_1"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-07-26T18:44:10.501Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjIwOTYxNjM5RjVGMDNFNDMxNwA=', 'content': 'Aprovecha esta oportunidad que BanCoppel tiene para ti, para que liquides tu cuenta, tenemos opciones para ti, estamos para apoyarte.', 'direction': 'outgoing', 'status': 'failed'}, {'createdAt': '2024-08-22T15:58:31.325Z', 'role': 'assistant', 'content': 'Template ""banco_nos_interesa_acuerdo"" enviado (Solo tú puedes ver este mensaje)', 'direction': 'outgoing'}, {'createdAt': '2024-08-22T15:58:31.325Z', 'role': 'assistant', 'id': 'wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjI5QkZDQjQ2Q0VFRUE0NkFDNgA=', 'content': 'Nos interesa llegar a un acuerdo para tu credito *BanCoppel*, estamos para ayudarte, ¿qué opinas?', 'direction': 'outgoing', 'status': 'failed'}]",MARIA,0,353257377876857,True,detener promociones,Duplicado
freq,19184,3,2,25171,19406,27494,24087,27439,25817,1,...,6189,3769,730,1,1684,7918,27494,26005,517,2


In [571]:
conversations = []

for index, item in tqdm(df_bancoppel.iterrows(), total=df_bancoppel.shape[0]):
    
    temp_item = []
    for k in item['messages']:
        if 'content' not in k.keys():
            continue
        if "Template" in k['content']:
            k['content'] = " " + k['content']  # Un espacio en blanco es el primer ascii
        temp_item.append(k)
    
    tag = item['tag'] if 'tag' in item.keys() else "sin_info_para_clasificar"
    
    rand_conversationid = random.randint(1e6, 1e8)
    
    for it in range(len(temp_item)):
        temp_item[it]['conversationId'] = rand_conversationid
        temp_item[it]['credit'] = item['credit']
        temp_item[it]['discount'] = item['discount']
        temp_item[it]['phone1'] = item['contact_phone']
        temp_item[it]['last_template_at'] = item['last_template_at']
        temp_item[it]['tag'] = tag
        
        temp_item[it]['send_status'] = item['status'] if 'status' in item.keys() else 'sin_estatus'
    
    # Añadimos los elementos procesados a la lista conversations
    conversations.extend(temp_item)

# Convertimos la lista de diccionarios a un DataFrame de pandas
conversations = pd.DataFrame(conversations)

100%|██████████| 27494/27494 [00:02<00:00, 13745.19it/s]


In [572]:
conversations.head()

,createdAt,role,content,direction,conversationId,credit,discount,phone1,last_template_at,tag,send_status,id,status,seen,image_id
0,2024-07-26T18:44:10.501Z,assistant,"Template ""banco_liquidacion_3_1"" enviado (Solo tú puedes ver este mensaje)",outgoing,11098050,600748049795,0,5212201140267,2024-08-22T15:58:31.325Z,NaN,rc,NaN,NaN,NaN,NaN
1,2024-07-26T18:44:10.501Z,assistant,"Aprovecha esta oportunidad que BanCoppel tiene para ti, para que liquides tu cuenta, tenemos opciones para ti, estamos para apoyarte.",outgoing,11098050,600748049795,0,5212201140267,2024-08-22T15:58:31.325Z,NaN,rc,wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjIwOTYxNjM5RjVGMDNFNDMxNwA=,failed,NaN,NaN
2,2024-08-22T15:58:31.325Z,assistant,"Template ""banco_nos_interesa_acuerdo"" enviado (Solo tú puedes ver este mensaje)",outgoing,11098050,600748049795,0,5212201140267,2024-08-22T15:58:31.325Z,NaN,rc,NaN,NaN,NaN,NaN
3,2024-08-22T15:58:31.325Z,assistant,"Nos interesa llegar a un acuerdo para tu credito *BanCoppel*, estamos para ayudarte, ¿qué opinas?",outgoing,11098050,600748049795,0,5212201140267,2024-08-22T15:58:31.325Z,NaN,rc,wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjI5QkZDQjQ2Q0VFRUE0NkFDNgA=,failed,NaN,NaN
4,2024-08-01T00:07:50.281Z,assistant,"Template ""banco_como_te_ayudamos"" enviado (Solo tú puedes ver este mensaje)",outgoing,27995135,600443166431,0,5212211180442,2024-08-10T14:48:24.586Z,NaN,rc,NaN,NaN,NaN,NaN


In [573]:
conversations.content.str.contains("Template").sum()

83662

In [574]:
conversations.drop("send_status", axis = 1, inplace=True)
conversations.rename(columns={"status": "sent_status"}, inplace = True)
conversations.sent_status.fillna("sin_estatus", inplace=True)

In [575]:
conversations.last_template_at = pd.to_datetime(conversations.last_template_at)

conversations.last_template_at = conversations.last_template_at.dt.tz_convert('America/Mexico_City')

conversations.last_template_at = conversations.last_template_at.dt.tz_localize(None)

In [576]:
conversations.createdAt = pd.to_datetime(conversations.createdAt)

conversations.createdAt = conversations.createdAt.dt.tz_convert('America/Mexico_City')

conversations.createdAt = conversations.createdAt.dt.tz_localize(None)

In [577]:
conversations.drop_duplicates(['createdAt', 'phone1', 'credit', 'content', 'role'], inplace=True)

In [578]:
conversations.reset_index(drop=True, inplace=True)

In [579]:
conversations

,createdAt,role,content,direction,conversationId,credit,discount,phone1,last_template_at,tag,id,sent_status,seen,image_id
0,2024-07-26 12:44:10.501,assistant,"Template ""banco_liquidacion_3_1"" enviado (Solo tú puedes ver este mensaje)",outgoing,11098050,600748049795,0,5212201140267,2024-08-22 09:58:31.325,NaN,NaN,sin_estatus,NaN,NaN
1,2024-07-26 12:44:10.501,assistant,"Aprovecha esta oportunidad que BanCoppel tiene para ti, para que liquides tu cuenta, tenemos opciones para ti, estamos para apoyarte.",outgoing,11098050,600748049795,0,5212201140267,2024-08-22 09:58:31.325,NaN,wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjIwOTYxNjM5RjVGMDNFNDMxNwA=,failed,NaN,NaN
2,2024-08-22 09:58:31.325,assistant,"Template ""banco_nos_interesa_acuerdo"" enviado (Solo tú puedes ver este mensaje)",outgoing,11098050,600748049795,0,5212201140267,2024-08-22 09:58:31.325,NaN,NaN,sin_estatus,NaN,NaN
3,2024-08-22 09:58:31.325,assistant,"Nos interesa llegar a un acuerdo para tu credito *BanCoppel*, estamos para ayudarte, ¿qué opinas?",outgoing,11098050,600748049795,0,5212201140267,2024-08-22 09:58:31.325,NaN,wamid.HBgNNTIxMjIwMTE0MDI2NxUCABEYEjI5QkZDQjQ2Q0VFRUE0NkFDNgA=,failed,NaN,NaN
4,2024-07-31 18:07:50.281,assistant,"Template ""banco_como_te_ayudamos"" enviado (Solo tú puedes ver este mensaje)",outgoing,27995135,600443166431,0,5212211180442,2024-08-10 08:48:24.586,NaN,NaN,sin_estatus,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194695,2024-08-21 12:28:38.859,assistant,"¡Excelente día! tenemos muy buenas alternativas para que liquides tu cuenta con *BanCoppel*, esperamos tu respuesta.",outgoing,94101357,600684070276,0,5219993630543,2024-08-21 12:28:38.859,NaN,wamid.HBgNNTIxOTk5MzYzMDU0MxUCABEYEjRBQzFDNTc3MDY1NENDNkEyOQA=,sin_estatus,NaN,NaN
194696,2024-08-21 12:23:44.487,assistant,"Template ""banco_tenemos_alternativas"" enviado (Solo tú puedes ver este mensaje)",outgoing,54625028,600657315930,0,5219994188914,2024-08-21 12:23:44.487,NaN,NaN,sin_estatus,NaN,NaN
194697,2024-08-21 12:23:44.487,assistant,"¡Excelente día! tenemos muy buenas alternativas para que liquides tu cuenta con *BanCoppel*, esperamos tu respuesta.",outgoing,54625028,600657315930,0,5219994188914,2024-08-21 12:23:44.487,NaN,wamid.HBgNNTIxOTk5NDE4ODkxNBUCABEYEjQ1NzQwNUVDNzU4RUY5NzlBRgA=,sin_estatus,NaN,NaN
194698,2024-08-21 12:26:16.227,assistant,"Template ""banco_tenemos_alternativas"" enviado (Solo tú puedes ver este mensaje)",outgoing,84375901,600747547997,0,5219996487237,2024-08-21 12:26:16.227,NaN,NaN,sin_estatus,NaN,NaN


In [580]:
conversations['tag'] = conversations['tag'].replace('no defineno defineno defineno defineno defineno defineno defineno defineno defineno define', 'no define')
print(conversations['tag'].unique())

[nan 'intención de pago' 'no define' 'carta convenio'
 'detener promociones' 'contacto humano' 'no reconoce' 'próximo pago'
 'negativa de pago']


In [581]:
conversations.tag.value_counts()

tag
carta convenio         7113
detener promociones    5230
no define              4610
contacto humano        3900
próximo pago           1982
no reconoce            1249
intención de pago       744
negativa de pago        142
Name: count, dtype: int64

In [582]:
tag_counts = conversations.tag.value_counts()
fig = px.pie(values=tag_counts.values, names = tag_counts.index, title = 'Distribución de tags en conversaciones', width=600, height=400)
fig.show()

In [583]:
conversations.columns

Index(['createdAt', 'role', 'content', 'direction', 'conversationId', 'credit',
       'discount', 'phone1', 'last_template_at', 'tag', 'id', 'sent_status',
       'seen', 'image_id'],
      dtype='object')

In [584]:
data = conversations.copy()

In [585]:
serie_count_celphones = pd.DataFrame(data.groupby("conversationId").phone1.nunique().sort_values()).reset_index()

In [586]:
data = data.loc[data.conversationId.isin(serie_count_celphones.query("phone1 == 1").conversationId)]

In [587]:
data.groupby("conversationId").phone1.nunique().sort_values().min() == data.groupby("conversationId").phone1.nunique().sort_values().max()

True

In [588]:
data.groupby("conversationId").credit.nunique().sort_values().min() == data.groupby("conversationId").credit.nunique().sort_values().max()

True

In [589]:
data.dropna(subset=["phone1", 'credit'], inplace=True)

In [590]:
data['id'] = data.apply(lambda row: f"{row['credit']} - {row['phone1']}", axis = 1)

In [591]:
## Filtrando solo las fechas solicitadas
data.credit = data.credit.astype(str)
data.conversationId = data.conversationId.astype(str)

In [592]:
data.reset_index(drop=True, inplace=True)

In [593]:
data = data.sort_values(by = ['createdAt', 'content'])
data = data.reset_index()

In [594]:
for i, row in data.iterrows():
    if "Template" in row['content']:
        data.at[i, "sent_status"] = data.at[i+1, "sent_status"]

In [595]:
dummies = pd.get_dummies(data['sent_status'], prefix='sent_status')

# Concatenar dummies con el DataFrame original excluyendo la columna original
data = pd.concat([data.drop('sent_status', axis=1), dummies], axis=1)

In [596]:
data

,index,createdAt,role,content,direction,conversationId,credit,discount,phone1,last_template_at,tag,id,seen,image_id,sent_status_delivered,sent_status_failed,sent_status_read,sent_status_sent,sent_status_sin_estatus
0,17970,2024-06-12 10:59:56.767,assistant,"Template ""descuento_banco_60"" enviado",outgoing,1284442,600411910950,0,5212411776482,2024-08-22 17:13:41.796,NaN,600411910950 - 5212411776482,NaN,NaN,False,False,False,False,True
1,17971,2024-06-12 10:59:56.767,assistant,"El dia de HOY tiene un magnifico descuento del 60% para liquidar su credito Bancoppel, ESPERAMOS SU RESPUESTA.",outgoing,1284442,600411910950,0,5212411776482,2024-08-22 17:13:41.796,NaN,600411910950 - 5212411776482,NaN,NaN,False,False,False,False,True
2,129557,2024-06-12 11:22:36.582,assistant,"Template ""descuento_banco_60"" enviado",outgoing,93421851,600718450494,0,5218445318517,2024-08-21 11:54:51.699,NaN,600718450494 - 5218445318517,NaN,NaN,False,False,False,False,True
3,129558,2024-06-12 11:22:36.582,assistant,"El dia de HOY tiene un magnifico descuento del 60% para liquidar su credito Bancoppel, ESPERAMOS SU RESPUESTA.",outgoing,93421851,600718450494,0,5218445318517,2024-08-21 11:54:51.699,NaN,600718450494 - 5218445318517,NaN,NaN,False,False,False,False,True
4,21282,2024-06-12 11:22:37.150,assistant,"Template ""descuento_banco_60"" enviado",outgoing,51774339,680087146097,0,5213315714166,2024-08-17 09:11:06.908,NaN,680087146097 - 5213315714166,NaN,NaN,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194655,166346,2024-08-26 18:37:59.690,assistant,Gracias por tu mensaje. Estamos conectándote con un asesor.,outgoing,8951569,600569368480,0,5215564764806,2024-08-22 10:36:51.183,no define,600569368480 - 5215564764806,False,NaN,False,False,True,False,False
194656,81768,2024-08-26 18:51:23.417,user,"Hola! Tengo adeudo, quiero abonar o ver que opciones tengo",incoming,93145663,600741435819,0,5216863208995,2024-08-21 14:53:50.148,no define,600741435819 - 5216863208995,NaN,NaN,False,False,False,False,True
194657,103887,2024-08-26 19:09:17.187,user,Ok 👍 si lo voy intentar,incoming,91209407,600628048214,0,5213334624563,2024-08-26 12:51:12.707,no define,600628048214 - 5213334624563,NaN,NaN,False,False,False,False,True
194658,81769,2024-08-26 19:11:37.763,assistant,"En sistema me aparece una TDC VISA Bancoppel con un saldo pendiente de 24,321.08 Puede liquidar con un pago único de $9,800 Este pago se realiza en una sola exhibición directamente en ventanilla de Bancoppel ¿Contaría con esta cantidad para el día miércoles 28 de agosto?",outgoing,93145663,600741435819,0,5216863208995,2024-08-21 14:53:50.148,no define,600741435819 - 5216863208995,NaN,NaN,False,False,True,False,False


In [597]:
respuestas = 0
indexes_templates = np.where(data.content.str.contains("Template"))[0]
for index in range(len(indexes_templates) - 1):
    if (indexes_templates[index + 1] - indexes_templates[index]) > 2:
        respuestas += 1

In [598]:
respuestas

3829

In [599]:
# cuenta la cantidad de trues y false en la columna sent_status_sent

data['sent_status_sent'].value_counts()

sent_status_sent
False    181664
True      12996
Name: count, dtype: int64

<font size = 8 color ='27A3D7'>Insights</font>

---

## 1. Envíos Totales



In [600]:
# conversations.role.value_counts()

In [601]:
data.query("content.str.contains('Template')").role.value_counts()

role
assistant    83647
Name: count, dtype: int64

## 2. Porcentaje de Respuestas



In [602]:
total_envios = data.query("content.str.contains('Template')").shape[0]
total_respuestas = data.query("direction == 'incoming'").shape[0]

total_envios, total_respuestas

(83647, 11574)

In [603]:
if total_envios > 0: 
    porcentaje_respuestas = (total_respuestas / total_envios) * 100
else:
    porcentaje_respuestas = 0

print(f"Porcentaje de respuestas recibidas: {porcentaje_respuestas:.2f}%")

Porcentaje de respuestas recibidas: 13.84%


## 3. Gastos en Meta

Mexico	$US	0.0436	0.01	0.0239	n/a	0.0105

https://developers.facebook.com/docs/whatsapp/pricing?locale=es_LA

In [604]:
import requests

def obtener_tipo_cambio():
    try:
        response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
        data = response.json()
        tipo_cambio_usd_mxn = data["rates"]["MXN"]
        return tipo_cambio_usd_mxn
    except Exception as e:
        print(f"Error al obtener el tipo de cambio: {e}")
        return 19.33  # Valor por defecto en caso de error

def calcular_costo_por_mes(data, mes, año):
    data['createdAt'] = pd.to_datetime(data['createdAt'])
    
    mensajes_template = data[data['content'].str.contains('Template', na = False) & (data['sent_status_failed'] != True)]

    mensajes_template_mes = mensajes_template[(mensajes_template['createdAt'].dt.month == mes) &
                                              (mensajes_template['createdAt'].dt.year == año)]

    total_envios = mensajes_template_mes.shape[0]
    tipo_cambio_usd_mxn = obtener_tipo_cambio()
    costo_usd_por_mensaje = 0.0436
    costo_mxn_por_mensaje = costo_usd_por_mensaje * tipo_cambio_usd_mxn
    total_a_pagar_mxn = total_envios * costo_mxn_por_mensaje

    print(f"Tipo de cambio USD a MXN: {tipo_cambio_usd_mxn:.2f}")
    print(f"Costo por mensaje en MXN: ${costo_mxn_por_mensaje:.4f}")
    print(f"Mes/Año: {mes}/{año}")
    print(f"Total de envíos realizados por el chatbot (sin fallos): {total_envios}")
    print(f"Total a pagar por los mensajes enviados: ${total_a_pagar_mxn:.2f} MXN")


In [605]:
calcular_costo_por_mes(data, 8, 2024)

Tipo de cambio USD a MXN: 19.38
Costo por mensaje en MXN: $0.8450
Mes/Año: 8/2024
Total de envíos realizados por el chatbot (sin fallos): 42402
Total a pagar por los mensajes enviados: $35828.33 MXN


In [606]:
total_envios = data.query("content.str.contains('Template') and sent_status_failed == False").shape[0]

tipo_cambio_usd_mxn = obtener_tipo_cambio()
costo_usd_por_mensaje = 0.0436
costo_mxn_por_mensaje = costo_usd_por_mensaje * tipo_cambio_usd_mxn
total_a_pagar_mxn = total_envios * costo_mxn_por_mensaje

print(f"Tipo de cambio USD a MXN: {tipo_cambio_usd_mxn:.2f}")
print(f"Costo por mensaje en MXN: ${costo_mxn_por_mensaje:.4f}")
print(f"Total de envíos realizados por el chatbot (sin fallos): {total_envios}")
print(f"Total a pagar por los mensajes enviados: ${total_a_pagar_mxn:.2f} MXN")

Tipo de cambio USD a MXN: 19.38
Costo por mensaje en MXN: $0.8450
Total de envíos realizados por el chatbot (sin fallos): 57531
Total a pagar por los mensajes enviados: $48611.85 MXN


<font size = 7 color ='27A3D7'>Dictaminaciones General</font>

---

de las respuestas: extraigo el tag y saco el el count, lo grafico

pie chart o donut?

In [607]:
tag_counts = data.query("content.str.contains('Template')").tag.value_counts()

fig = px.pie(
    values=tag_counts.values,
    names=tag_counts.index,
    title='Distribución de Tags',
    width=600,
    height=400,
    hole=0.4  
)

fig.show()

In [608]:
tag_counts = data.query("content.str.contains('Template')").tag.value_counts()
fig = px.pie(values = tag_counts.values, names = tag_counts.index, title = 'Distribución de Tags', width = 600, height = 400)
fig.show()

<font size = 7 color ='27A3D7'>Funnel de Envíos</font>

---

In [609]:
filtered_data = data.query("content.str.contains('Template')")
true_counts = filtered_data[['sent_status_delivered', 'sent_status_failed', 'sent_status_read', 'sent_status_sent', 'sent_status_sin_estatus']].apply(pd.value_counts).loc[True]
fig = px.pie(values=true_counts, names=true_counts.index, title='Distribución de "True"', width = 600, height = 400)
fig.show()


In [610]:
respuestas

3829

In [611]:
filtered_data = data.query("content.str.contains('Template')")

response_count = total_respuestas # o respuestas
read_count = filtered_data['sent_status_read'].sum() 
delivered_count = filtered_data['sent_status_delivered'].sum() + read_count
sent_count = filtered_data['sent_status_sent'].sum() + read_count + delivered_count

values = [sent_count, delivered_count, read_count, response_count]
stages = ['Envíados', 'Recibidos', 'Leídos', 'Respondidos']

fig = go.Figure(go.Funnel(
    y=stages,
    x=values,
    textinfo="value+percent initial"
))

fig.update_layout(title='Funnel de Envíos')
fig.show()


<font size = 7 color ='27A3D7'>Tabla de Filtros</font>

---

Descargar cuales reintentar, persona (ID) para identificar, tag, cantidad de envios. Fecha de último envío, status del último envío. Binaria, bloquado o no.

Filtros en ST: Tag, (pe. intención de pago, no definen) Fecha, (que fechas es bueno descargar), Cantidad de Envios (Menor a X envíos), Cantidad de Respuestas, Eficiencia de Respuestas (res/envios)

In [612]:
session = create_aws_session()
s3_client = session.client('s3')

bucket_name = 's3-pernexium-report'
prefix = 'master/bancoppel/reportes/chatbot/2024_08/'

response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    xlsx_files = [
        obj for obj in response['Contents'] if obj['Key'].endswith('.xlsx')
    ]
    xlsx_files = sorted(xlsx_files, key=lambda x: x['LastModified'], reverse=True)

    latest_file = xlsx_files[0]['Key']

    file_object = s3_client.get_object(Bucket=bucket_name, Key=latest_file)
    file_content = file_object['Body'].read()

    df_filtros = pd.read_excel(BytesIO(file_content))

    print(df_filtros)
else:
    print("No se encontraron archivos en el bucket con el prefijo especificado.")


            credito  telefono_contactado  conversation_id         segmento  \
0      680078677837           2722365544         94133230  pendiente cruce   
1      680078677837           2722365544         32080612  pendiente cruce   
2      680061459128           2281243276         17118430  pendiente cruce   
3      770007726811           7712016720         20471243  pendiente cruce   
4      680081720814           3318855860          1838854  pendiente cruce   
...             ...                  ...              ...              ...   
27318  600545196195           2323263372         50814490  pendiente cruce   
27319  600656572168           5566540506         96342246  pendiente cruce   
27320  600345050972           8677511848          8947323  pendiente cruce   
27321  600575541096           3121216456          9319486  pendiente cruce   
27322  600643323600           7341315206         49362820  pendiente cruce   

       ultima_respuesta_cliente  contesta_despues_ofrecer_carta

In [613]:
df_filtros.columns

Index(['credito', 'telefono_contactado', 'conversation_id', 'segmento',
       'ultima_respuesta_cliente', 'contesta_despues_ofrecer_carta',
       'respuesta_cliente_despues_ofrecer_carta', 'ultimo_envio_estatus',
       'ultima_respuesta_cliente_periodo', 'detonaciones_enviadas_periodo',
       'detonaciones_vistas_periodo', 'detonaciones_entregadas_periodo',
       'detonaciones_fallidas_periodo',
       'detonaciones_aceptadas_por_meta_periodo',
       'detonaciones_sin_estatus_periodo', 'total_respuestas_periodo',
       'eficiencia_respuestas_periodo', 'primera_detonacion_periodo',
       'ultima_detonacion_periodo', 'ha_respondido_periodo', 'status_periodo',
       'chatbot_ofrece_carta_convenio_periodo', 'periodo 7 a 8',
       'periodo 8 a 9', 'periodo 9 a 10', 'periodo 10 a 11', 'periodo 11 a 12',
       'periodo 12 a 13', 'periodo 13 a 14', 'periodo 14 a 15',
       'periodo 15 a 16', 'periodo 16 a 17', 'periodo 17 a 18',
       'periodo 18 a 19', 'periodo 19 a 20', 'periodo

In [614]:
df_filtros.drop(['segmento'], axis=1, inplace=True) # agregar que columnas no se requieren

In [615]:
df_filtros

,credito,telefono_contactado,conversation_id,ultima_respuesta_cliente,contesta_despues_ofrecer_carta,respuesta_cliente_despues_ofrecer_carta,ultimo_envio_estatus,ultima_respuesta_cliente_periodo,detonaciones_enviadas_periodo,detonaciones_vistas_periodo,...,trim 13 a 14,trim 14 a 15,trim 15 a 16,trim 16 a 17,trim 17 a 18,trim 18 a 19,trim 19 a 20,trim 20 a 21,trim 21 a 22,ultima_hora_respuesta_cliente_trim
0,680078677837,2722365544,94133230,Dame un buen descuento,False,No responde,read,2024-08-01,2,2,...,0,0,0,0,0,0,0,0,0,NaN
1,680078677837,2722365544,32080612,Sin respuesta,False,No responde,sin_estatus,NaT,1,0,...,0,0,0,0,0,0,0,0,0,NaN
2,680061459128,2281243276,17118430,Necesitaría tres pagos,False,No responde,read,2024-08-01,2,1,...,0,0,0,0,0,0,0,0,0,NaN
3,770007726811,7712016720,20471243,Cuéntame del descuento,False,No responde,read,2024-08-15,3,3,...,0,0,0,0,0,0,0,0,0,NaN
4,680081720814,3318855860,1838854,Ok,False,No responde,read,2024-08-01,3,3,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,600545196195,2323263372,50814490,Sin respuesta,False,No responde,read,NaT,1,1,...,0,0,0,0,0,0,0,0,0,NaN
27319,600656572168,5566540506,96342246,Gracias,False,No responde,read,2024-08-23,1,1,...,0,0,0,0,0,0,0,0,0,NaN
27320,600345050972,8677511848,8947323,Sin respuesta,False,No responde,read,NaT,1,1,...,0,0,0,0,0,0,0,0,0,NaN
27321,600575541096,3121216456,9319486,Sin respuesta,False,No responde,sent,NaT,1,0,...,0,0,0,0,0,0,0,0,0,NaN


<font size = 7 color ='27A3D7'>Cantidad de Respuestas</font>

---

In [616]:
dias_respuestas = []

for _, row in df_bancoppel.iterrows():
    try:
        if isinstance(row['messages'], str):
            messages = json.loads(row['messages'])
        elif isinstance(row['messages'], list):
            messages = row['messages']
        else:
            continue 
        
        for message in messages:
            if message.get('role') != 'assistant' and message.get('direction') == 'incoming':
                created_at = message.get('createdAt')
                if created_at:
                    # Convertimos a datetime y restamos 6 horas
                    adjusted_time = pd.to_datetime(created_at) - pd.Timedelta(hours=6)
                    day_of_week = adjusted_time.day_name()
                    dias_respuestas.append(day_of_week)

    except (json.JSONDecodeError, TypeError) as e:
        print(f"Error al procesar la fila: {e}")

df_dias_respuestas = pd.DataFrame(dias_respuestas, columns=['Día de la semana'])

conteo_respuestas = df_dias_respuestas['Día de la semana'].value_counts().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], fill_value=0)


fig = px.bar(conteo_respuestas, x=conteo_respuestas.index, y=conteo_respuestas.values,
             labels={'x': 'Día de la semana', 'y': 'Cantidad de respuestas'},
             title='Cantidad de Respuestas por Día de la Semana')

fig.show()


In [617]:
import pytz

horas_dias_respuestas = []
timezone_cdmx = pytz.timezone('America/Mexico_City')

for _, row in df_bancoppel.iterrows():
    try:
        if isinstance(row['messages'], str):
            messages = json.loads(row['messages'])
        elif isinstance(row['messages'], list):
            messages = row['messages']
        else:
            continue 
        
        for message in messages:
            if message.get('role') != 'assistant' and message.get('direction') == 'incoming':
                created_at = message.get('createdAt')
                if created_at:
                    dt = pd.to_datetime(created_at).tz_convert('UTC')  # Convertimos a UTC
                    dt_cdmx = dt.tz_convert(timezone_cdmx)  # Convertimos de UTC a la zona horaria de CDMX
                    day_of_week = dt_cdmx.strftime('%A')
                    hour_of_day = dt_cdmx.hour
                    horas_dias_respuestas.append((day_of_week, hour_of_day))

    except (json.JSONDecodeError, TypeError) as e:
        print(f"Error al procesar la fila: {e}")

df_horas_dias_respuestas = pd.DataFrame(horas_dias_respuestas, columns=['Día de la semana', 'Hora del día'])

df_horas_dias_respuestas['Día de la semana'] = df_horas_dias_respuestas['Día de la semana'].replace({
    'Monday': 'lunes',
    'Tuesday': 'martes',
    'Wednesday': 'miércoles',
    'Thursday': 'jueves',
    'Friday': 'viernes',
    'Saturday': 'sábado',
    'Sunday': 'domingo'
})

heatmap_data = df_horas_dias_respuestas.pivot_table(index='Día de la semana', columns='Hora del día', aggfunc='size', fill_value=0)

heatmap_data = heatmap_data.reindex(["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"])

fig = px.imshow(heatmap_data, labels=dict(x="Hora del día", y="Día de la semana", color="Cantidad de respuestas"),
                x=heatmap_data.columns, y=heatmap_data.index,
                title="Mapa de Calor de Respuestas por Hora del Día y Día de la Semana")

fig.show()


<font size = 7 color ='27A3D7'>Mapa</font>

---